# BUAD 313 - Spring 2025 - Assignment 1 (100 points)

Notes:
 - You may work in teams of up to 3.  Submit one assignment for the three of you, but please ensure it has all 3 of your names and @usc.edu emails.
 - You must submit your work as a .ipynb file (jupyter notebook). The grader has to be able to run your notebook. Code that doesn't run gets zero points.  You may also submit a .pdf version if you wish.  
 - Use the existing sections below to submit your answers below.  You can add additional Python/markdown cells to describe and explain your solution, but keep it tidy.

The deadline for this assignment is **11:59 PM Pacific Time on Friday February 7, 2024**. Late submissions will not be accepted.

Below are the standard Python packages that we use for optimization models in this course. By running this next Python cell, you will have these packages available to use in all your answers contained in this file.

In [1]:
import numpy as np
from gurobipy import Model, GRB, quicksum

## Team Names and Emails:
 <font color="blue">**(Edit this cell)**</font>
 - Sumin Wu suminwu@usc.edu
 - Raptor English rmenglis@usc.edu

## Question 1 (36 Points)

Trojan Kicks manufactures kick scooters. The firm’s assembly process has four workstations. See the process flow diagram below.

<img src="trojanScootersFlow.png" width="1000" />

*Note that if you do not have the file trojanScootersflow.png in the same folder or directory as this notebook, the image will not display. Displaying the figure is not necessary for receiving full credit on this assignment.*


- **Station A** makes the decks; at this workstation, decks are stamped out, welded, and painted.
- At **Station B**, workers assemble the fork, steer support, and T-handle.
- At **Station C**, workers assemble the wheel, brake, and steering mechanism.
- At **Station D**, workers apply decals and grip tape, and conduct the final functional test.

The firm manufactures three products that compete in different segments of the market—**Trojan** scooters in the premium high-end segment, **Tommy** scooters in the mid-market segment, and  **TK** scooters in the low-end segment.

The decks for TK are outsourced; hence, **Station A produces decks only for two products**—Trojan and Tommy. Processing times (in minutes per unit) and the number of workers at each workstation are given in the table below. In each workstation, each worker works independently on their own scooter. The firm operates **8 hours a day, 5 days a week**.  Notice the last column gives the demand for each type of scooter.

| **Product**          | **Station A** (mins/unit) | **Station B** (mins/unit) | **Station C** (mins/unit) | **Finishing** (mins/unit) | **Demand (scooters/week)** |
|----------------------|-------------------------|-------------------------|-------------------------|-------------------------|-------------------------|
| **Trojan**          | 20                      | 15                      | 19                      | 28                      | 150                     |
| **Tommy**           | 16                      | 12                      | 13                      | 22                      | 180                     |
| **TK**              | -                        | 10                      | 11                      | 21                      | 120                     |
| *Number of workers* | **3**                    | **5**                    | **6**                    | **4**                    | **N/A**                 |


Given their different marketing, different scooters earn different revenue:

| **Product** | Trojan | Tommy | TK  |
|------------|--------|--------|-----|
| **Price ($)** | 200    | 150    | 100 |


Your goal is to decide how many scooters of  each type to manufacture in a given week to maximize revenue. Note, you are NOT obligated to serve all demand.

**Teaching Note:  Note, this is a substantively more interesting version of a process analysis problem with multiple product types with different flows through the system and different demands.  The point of this problem is to show you how you can use linear optimization to tackle such process analysis questions**

### Part a) (5 points)
What are the decision variables in this this problem?  (Add a Markdown Cell directly below this cell with your answer).  Describe your decision variables in math and in words, and include units.  

M: Number of Tommy scooters produced per week (units: scooters/week)

T: Number of Trojan scooters produced per week (units: scooters/week)

K: Number of TK scooters produced per week (units: scooters/week)

A_w: Number of workers in Station A (units: workers)

B_w: Number of workers in Station B (units: workers)

C_w: Number of workers in Station C (units: workers)

F_w: Number of workers in Final Assembly (units: workers)

P_t: processing time for Trojan scooters (units: mins/unit)

P_m: processing time for Tommy scooters (units: mins/unit)

P_k: processing time for TK scooters (units: mins/unit)

### Part b) (5 points):

What is the objective function for this problem? (Add a markdown cell directly below this oen with your answer.) Describe the objective function both in words and in math.  include units.

How many Tommy scooters, Trojan scooters, and TK scooters to produce in a given week to maximize revenue. (scooters/week)

Maximize(200\*T + 150\*M + 100\*K)

### Part c) (10 points)
What are the constraints for this problem? For each constraint, be sure to include a short description (in words) of what the constraint represents.  Your description should make the units clear. You should also relax any integrality constraints (i.e. you **can** make fractional scooters since we're thinking about the long-term steady-state performance of the faculty.)

Demand Constraints: Production of each scooter should not exceed the scooter units demanded

M≤180 Tommy scooters demand constraint: max 180 scooters/week

T≤150 Trojan scooters demand constraint: max 150 scooters/week

K≤120 TK scooters demand constraint: max 120 scooters/week

Non-Negativity Constraints: Cannot produce negative scooters.

M≥0 (Non-negative production of Tommy scooters)  min 0 scooters/week

T≥0 (Non-negative production of Trojan scooters) min 0 scooters/week

K≥0 (Non-negative production of TK scooters) min 0 scooters/week

Station X Time Constraints: Total processing time at Station X cannot exceed worker availability for Station X.

Station A has 3 workers available for 40 hours per week, or 14,400 minutes per week. 16M+20T+0K≤3×5×480

Station B has 5 workers available for 40 hours per week, or 24,000 minutes per week. 12M+15T+10K≤5×5×480

Station C has 6 workers available for 40 hours per week, or 28,800 minutes per week. 13M+19T+11K≤6×5×480

### Part d) (10 points)
Using your formulation above, code up your model in Gurobi and solve it.  Write the optimal value and optimal solution in a markdown cell direclty below this one.  Be sure to label which is which and what the units are!

Below that markdown cell, include your python code for the model in one or more python cells.  Your code should print out the optimal value and optimal solution from your model as its last step. Code that does not run earns no credit!

Optimal Solution:

Produce 150.0 units of Trojan scooters per week

Produce 180.0 units of Tommy scooters per week

Produce 68.57142857142857 units of TK scooters per week

Optimal Value [Total Revenue]: 63857.142857142855 dollars

In [2]:
# define a gurobipy model m and name it "scooters"
m = Model("scooters")

# add linear decision variables for the inputs of creating the Tommy scooter

M = m.addVar(name="M", vtype=GRB.CONTINUOUS)
T = m.addVar(name="T", vtype=GRB.CONTINUOUS)
K = m.addVar(name="K", vtype=GRB.CONTINUOUS)

#create variables below:
A_w = m.addVar(name="A_w", vtype=GRB.CONTINUOUS)
B_w = m.addVar(name="B_w", vtype=GRB.CONTINUOUS)
C_w = m.addVar(name="C_w", vtype=GRB.CONTINUOUS)
F_w = m.addVar(name="F_w", vtype=GRB.CONTINUOUS)

P_t = m.addVar(name="P_T", vtype=GRB.CONTINUOUS)
P_m = m.addVar(name="P_M", vtype=GRB.CONTINUOUS)
P_k = m.addVar(name="P_K", vtype=GRB.CONTINUOUS)

Set parameter Username
Set parameter LicenseID to value 2609531
Academic license - for non-commercial use only - expires 2026-01-14


In [3]:
#add demand contraints, production of each scooter should not exceed the scooter units demanded
m.addConstr(M <= 180, "M_d")
m.addConstr(T <= 150, "T_d")
m.addConstr(K <= 120, "K_d")

#account for non-negativity, cannot make negative units of scooters
m.addConstr(M >= 0, "M_d")
m.addConstr(T >= 0, "T_d")
m.addConstr(K >= 0, "K_d")

#determine how many minutes are in a day the firm operates
mins_per_day = 8 * 60
mins_per_wk =  5 * mins_per_day

#station A, time constraint. Minutes to make scooters at station A must not exceed the amount of time the workers are available for station A
A_workers = 3
A_time = A_workers * mins_per_wk
m.addConstr(A_time >= 16 * M + 20 * T + 0 * K, "A_time")

#station B, time constraint. Minutes to make scooters at station B must not exceed the amount of time the workers are available for station B
B_workers = 5
B_time = B_workers * mins_per_wk
m.addConstr(15 * T + 12 * M + 10 * K <= B_time, name="Station_B_Time")

# station C, time constraint. Minutes to make scooters at station C must not exceed the amount of time the workers are available for station C
C_workers = 6
C_time = C_workers * mins_per_wk
m.addConstr(19 * T + 13 * M + 11 * K <= C_time, name="Station_C_Time")

# station F, time constraint. Minutes to make scooters at station F must not exceed the amount of time the workers are available for station F
F_workers = 4
F_time = F_workers * mins_per_wk
m.addConstr(28 * T + 22 * M + 21 * K <= F_time, name="Station_F_Time")

<gurobi.Constr *Awaiting Model Update*>

In [4]:
#set the objective function to maimize 200*T + 150*M + 100*K 
m.setObjective(200*T + 150*M + 100*K, GRB.MAXIMIZE)
m.optimize()
m.display()
if m.status == GRB.OPTIMAL:
    print("Optimal Solution")
    print(f'Produce {T.x} units of Trojan')
    print(f'Produce {M.x} units of Tommy')
    print(f'Produce {K.x} units of TK')
    print(f'Optimal Value [Total Revenue]: {m.objVal}')
else:
    print("No optimal")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.4.0 23E224)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 10 rows, 10 columns and 17 nonzeros
Model fingerprint: 0x7eb257b2
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [1e+02, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 1e+04]
Presolve removed 9 rows and 7 columns
Presolve time: 0.01s
Presolved: 1 rows, 3 columns, 3 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.8571429e+04   1.928571e+02   0.000000e+00      0s
       1    6.3857143e+04   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.02 seconds (0.00 work units)
Optimal objective  6.385714286e+04
Maximize
  150.0 M + 200.0 T + 100.0 K
Subject To
  M_d: M <= 180
  T_d: T <= 150
  K_d: K <= 120
  M_d: M >= 0
  T_d: T >= 0
  K_d: K >= 0
  A_time: 16.0 M + 20.0 T <= 7200
  St

/var/folders/x3/7wy7pwf915n6zxbg1rtvvclm0000gn/T/ipykernel_28371/594495953.py:4: DeprecationWarning: Model.display() is deprecated
  m.display()


### part d) (6 points)
What are the tight constraints at optimality?  (Hint: You'd expect that there should be 3 of them...)

How would you use these to communicate the optimal solution to a non-technical stakeholder?

Tight constraints: Station F Time, Tommy Demand, Trojan demand 

The number of tommy scooters produced should not exceed 180 and the number of trojan scooters produced should not exceed 150. If we produce beyond these numbers, it is unclear how production processes and revenue will be affected. Just like how in class with the cookie example, the oven needs to always be running. In this example, Station F should always be running and workers should always to working at Station F, there shouldn't be any downtime. Stations A,B,C can be underused and still produce the same amount of revenue

In [5]:
# Check which constraints are tight
import numpy as np
num_tight_constr = 0
for constr in m.getConstrs():
    if np.abs(constr.slack) <= 1e-6:
        print(f"Tight constraint: {constr.constrName}")
        num_tight_constr +=1

Tight constraint: M_d
Tight constraint: T_d
Tight constraint: Station_F_Time


## Question 2 ( 30 Points ):  Fulfillment at "Tamazon"

We'll consider a stylized fulfillment problem, similar to the one Amazon solves every day to meet 2 day shipping guarantees for prime members.  

Tamazon has 3 principal warehouses in Los Angeles described in the table below.  With the debut of the new season of Squid Games, green track suits are far and away the most popular item on the site.  Each of the warehouses has a limited supply of track suits (one size fits most) listed as "Inventory" in the table below.  

| Warehouse     | Latitude | Longitude  | Inventory |
|------------------|----------|------------|-----------|
| Hollywood        | 34.0928  | -118.3287  | 600      |
| Venice           | 33.9850  | -118.4695  | 300      |
| Downtown LA      | 34.0407  | -118.2468  | 200      |

There are hundreds of customers who have ordered the track suits and expect them to be delivered in the next two days.  The "fulfillment problem" is to decide from which warehosue we should ship the tracksuit to each customer in order to minimize total costs.  

To get some intuiton and make things simpler, your data science team clustered customer demand and identified 5 zones where most of the demand comes from.  

| Zone       | Latitude | Longitude  | Demand |
|--------------------|----------|------------|--------|
| Beverly Hills      | 34.0736  | -118.4004  | 250    |
| Santa Monica       | 34.0195  | -118.4912  | 150    |
| Westwood           | 34.0635  | -118.4455  | 200    |
| Silver Lake        | 34.0869  | -118.2702  | 310    |
| Echo Park          | 34.0782  | -118.2606  | 50     |

Shipping costs between places in LA is a bit tricky, especially with LA Traffic.  To make things simpler for now, let's assume that shipping costs per track suit are proportional to the distance between two points.  I did this calculation for you, so you don't have to code it.  The distances (in km) are given in the following table:

| Warehouse \ Zone   | Beverly Hills | Santa Monica | Westwood | Silver Lake | Echo Park |
|-------------|--------------:|-------------:|---------:|------------:|----------:|
| Hollywood   |          6.94 |        17.05 |    11.24 |        5.43 |      6.48 |
| Venice      |         11.73 |         4.33 |     9.00 |       21.58 |     21.86 |
| Downtown LA |         14.62 |        22.64 |    18.48 |        5.57 |      4.36 |

Remember, your goal is to ship track suits from the warehouses to the demand zones at minimal cost.


### Part a) Decision Variables (5 points)
What are the decision variables in this this problem? (Add a Markdown Cell directly below this cell with your answer). Describe your decision variables in math and in words, and include units.

Let H_B = Inventory Shipment from Hollywood to Beverly Hills (item units)

Let H_SM = Inventory Shipment from Hollywood to Santa Monica (item units)

Let H_SL = Inventory Shipment from Hollywood to Silver Lake (item units)

Let H_W = Inventory Shipment Hollywood to Westwood (item units)

Let H_E = Inventory Shipment Hollywood to Echopark (item units)

Let V_B = Inventory Shipment Venice to Beverly Hills (item units)

Let V_SM = Inventory Shipment Venice to Santa Monica (item units)

Let V_SL = Inventory Shipment Venice to Silver Lake (item units)

Let V_W = Inventory Shipment Venice to Westwood(item units)

Let V_E = Inventory Shipment Venice to Echopark (item units)

Let D_B = Inventory Shipment Downtown LA to Beverly Hills(item units)

Let D_SM = Inventory Shipment Downtown LA to Santa Monica (item units)

Let D_SL = Inventory Shipment Downtown LA to Silver Lake(item units)

Let D_W = Inventory Shipment Downtown LA to Westwood(item units)

Let D_E = Inventory Shipment Downtown LA to Echopark (item units)

### Part b) (5 points)
What is the objective function for this problem? (Add a markdown cell directly below this one with your answer.) Describe the objective function both in words and in math. include units.

We aim to minimize the total shipping cost, which is proportional to the total distance traveled weighted by the number of tracksuits shipped.

It costs 6.94 to ship from Hollywood to Beverly Hills.

It costs 17.05 to ship from Hollywood to Santa Monica.

It costs 11.24 to ship from Hollywood to Westwood.

It costs 5.43 to ship from Hollywood to Silver Lake.

It costs 6.48 to ship from Hollywood to Echo Park.

It costs 11.73 to ship from Venice to Beverly Hills.

It costs 4.33 to ship from Venice to Santa Monica.

It costs 9.00 to ship from Venice to Westwood.

It costs 21.58 to ship from Venice to Silver Lake.

It costs 21.86 to ship from Venice to Echo Park.

It costs 14.62 to ship from Downtown LA to Beverly Hills.

It costs 22.64 to ship from Downtown LA to Santa Monica.

It costs 18.48 to ship from Downtown LA to Westwood.

It costs 5.57 to ship from Downtown LA to Silver Lake.

It costs 4.36 to ship from Downtown LA to Echo Park.

Objective Function: Minimize(6.94 * H_B + 17.05 * H_SM + 11.24 * H_W + 5.43 * H_SL + 6.48 * H_E + 11.73 * V_B + 4.33 * V_SM + 9.00 * V_W + 21.58 * V_SL + 21.86 * V_E + 14.62 * D_B + 22.64 * D_SM + 18.48 * D_W + 5.57 * D_SL + 4.36 * D_E)

### Part c) (10 points)
What are the constraints for this problem? For each constraint, be sure to include a short description (in words) of what the constraint represents. Your description should make the units clear. You should also relax any integrality constraints (i.e. you can ship fractional track suits.)


Each warehouse has a limited supply capacity. The total number of units allocated from each warehouse to demand areas **cannot exceed** its supply limit.

Let:
- \( H_X \) represent the units allocated from **Hollywood** to demand area \( X \).
- \( V_X \) represent the units allocated from **Venice** to demand area \( X \).
- \( D_X \) represent the units allocated from **Downtown LA** to demand area \( X \).
- For x $\in$ {B, SM, W, SL, E}

$$
H_B + H_{SM} + H_W + H_{SL} + H_E \leq 600
$$

$$
V_B + V_{SM} + V_W + V_{SL} + V_E \leq 300
$$

$$
D_B + D_{SM} + D_W + D_{SL} + D_E \leq 200
$$


Each demand area has a **fixed demand** that must be **fully met** by supply from any combination of the warehouses.

$$
H_B + V_B + D_B = 250 \quad \text{(Beverly Hills demand)}
$$

$$
H_{SM} + V_{SM} + D_{SM} = 150 \quad \text{(Santa Monica demand)}
$$

$$
H_W + V_W + D_W = 200 \quad \text{(Westwood demand)}
$$

$$
H_{SL} + V_{SL} + D_{SL} = 310 \quad \text{(Silver Lake demand)}
$$

$$
H_E + V_E + D_E = 50 \quad \text{(Echo Park demand)}
$$

### Part d) (10 points)
Using your formulation above, code up your model in Gurobi and solve it. Write the optimal value and optimal solution in a markdown cell directly below this one. Be sure to label which is which and what the units are!

Below that markdown cell, include your python code for the model in one or more python cells. Your code should print out the optimal value and optimal solution from your model as its last step. Code that does not run earns no credit!

In [6]:
#create new model
m2 = Model("amazon")

#create variables
H_B = m2.addVar(name="H_B", vtype=GRB.CONTINUOUS, lb=0)
H_SM = m2.addVar(name="H_SM", vtype=GRB.CONTINUOUS, lb=0)
H_SL = m2.addVar(name="H_SL", vtype=GRB.CONTINUOUS, lb=0)
H_W = m2.addVar(name="H_W", vtype=GRB.CONTINUOUS, lb=0)
H_E = m2.addVar(name="H_E", vtype=GRB.CONTINUOUS, lb=0)

V_B = m2.addVar(name="V_B", vtype=GRB.CONTINUOUS, lb=0)
V_SM = m2.addVar(name="V_SM", vtype=GRB.CONTINUOUS, lb=0)
V_SL = m2.addVar(name="V_SL", vtype=GRB.CONTINUOUS, lb=0)
V_W = m2.addVar(name="V_W", vtype=GRB.CONTINUOUS, lb=0)
V_E = m2.addVar(name="V_E", vtype=GRB.CONTINUOUS, lb=0)

D_B = m2.addVar(name="D_B", vtype=GRB.CONTINUOUS, lb=0)
D_SM = m2.addVar(name="D_SM", vtype=GRB.CONTINUOUS, lb=0)
D_SL = m2.addVar(name="D_SL", vtype=GRB.CONTINUOUS, lb=0)
D_W = m2.addVar(name="D_W", vtype=GRB.CONTINUOUS, lb=0)
D_E = m2.addVar(name="D_E", vtype=GRB.CONTINUOUS, lb=0)

#add contraints for Inventory
m2.addConstr(H_B + H_SM + H_SL + H_W + H_E <= 600, "H_inventory")
#add contraints for Inventory for V
m2.addConstr(V_B + V_SM + V_SL + V_W + V_E <= 300, "V_inventory")
#add contraints for Inventory for D
m2.addConstr(D_B + D_SM + D_SL + D_W + D_E <= 200, "D_inventory")

#add contraints for Demand for B
m2.addConstr(D_B+V_B+H_B ==250, "B_Demand")
#add contraints for Demand for SM
m2.addConstr(D_SM+V_SM+H_SM ==150, "SM_Demand")
#add contraints for Demand for SL
m2.addConstr(D_SL+V_SL+H_SL ==310, "SL_Demand")
#add contraints for Demand for W
m2.addConstr(D_W+V_W+H_W ==200, "W_Demand")
#add contraints for Demand for E
m2.addConstr(D_E+V_E+H_E ==50, "E_Demand")

# create objective function
m2.setObjective(
    6.94 * H_B + 17.05 * H_SM + 11.24 * H_W + 5.43 * H_SL + 6.48 * H_E +
    11.73 * V_B + 4.33 * V_SM + 9.00 * V_W + 21.58 * V_SL + 21.86 * V_E +
    14.62 * D_B + 22.64 * D_SM + 18.48 * D_W + 5.57 * D_SL + 4.36 * D_E,
    GRB.MINIMIZE
)

# optimize model
m2.optimize()

# print results
if m2.status == GRB.OPTIMAL:
    print("Optimal solution found:")
    print(f'Hollywood -> Beverly: {H_B.x} units')
    print(f'Hollywood -> Santa Monica: {H_SM.x} units')
    print(f'Hollywood -> Westwood: {H_W.x} units')
    print(f'Hollywood -> Silver Lake: {H_SL.x} units')
    print(f'Hollywood -> Echo Park: {H_E.x} units')

    print(f'Venice -> Beverly: {V_B.x} units')
    print(f'Venice -> Santa Monica: {V_SM.x} units')
    print(f'Venice -> Westwood: {V_W.x} units')
    print(f'Venice -> Silver Lake: {V_SL.x} units')
    print(f'Venice -> Echo Park: {V_E.x} units')

    print(f'Downtown LA -> Beverly: {D_B.x} units')
    print(f'Downtown LA -> Santa Monica: {D_SM.x} units')
    print(f'Downtown LA -> Westwood: {D_W.x} units')
    print(f'Downtown LA -> Silver Lake: {D_SL.x} units')
    print(f'Downtown LA -> Echo Park: {D_E.x} units')

    print(f'Total cost: {m2.objVal}')
else:
    print("No optimal solution found.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.4.0 23E224)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 8 rows, 15 columns and 30 nonzeros
Model fingerprint: 0x12643152
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+01, 6e+02]
Presolve time: 0.00s
Presolved: 8 rows, 15 columns, 30 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.0858000e+03   5.000000e+01   0.000000e+00      0s
       2    6.1992000e+03   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.00 seconds (0.00 work units)
Optimal objective  6.199200000e+03
Optimal solution found:
Hollywood -> Beverly: 250.0 units
Hollywood -> Santa Monica: 0.0 units
Hollywood -> Westwood: 50.0 units
Hollywood -> Silver Lake: 300.0 units
Hollywood -> Echo Park: 0.0 units
Venice -> Beve

## Question 3 ( Points 34 pts ) : Cooling a Server Farm

This question is meant to push  your modeling abilities.  I believe in you.   Work together.  Think. Struggle.  Grow. :)

One of the unforeseen challenges in scaling up computing has been the massive envergy requirements of server farms and compute clusters.  A big portion of these requirements come from the need to cool the room (so the servers don't overheat).  But as they're working, the compute servers generate heat themselves.  We'll look at a stylized model to think about how to minimize cooling costs for a server farm.  We will focus for simplicity on an 8 hour day (although real farms run around the clock!).  In what follows, you should think like a physicist and interpret "cooling" as "negative heat."

In the absence of any other effects, we would expect that the temperature at the beginning of an hour would be the same as the temperature at the beginning of the previous hour.  We will say that to operate effecively, the server room must remain at or below 25 degrees Celsius at all times.  Notice the day starts at 9 am with a temperature of $22$ degrees Celsius.

That said, there are a few effects we should model:
  - During the  hour, the compute servers will do work.  As they work they generate additional heat.
  - During the hour, the ambient temperature provide additional heat (if it's hotter outside) or reduce the heat (if it's colder outside).  
  - During the hour, any cooling we apply (e.g. by cranking up the AC) will reduce the heat in the room.

In theory, we should model the precise temperature in the server room at all points in time (i.e. continuously).  As an approximation, we're only going to think about the temperature in the room at the beginning of every hour (starting at 9 am, and including 4:00 pm) and also at the end of the day (5:00 pm).  We're also going to assume that any additional heat generated or removed *during* the hour doesn't affect the temperature now, but affects the temperature in the next hour.


#### Server Work

Suppose that the demand for the server farm (in Teraflops/hr or TFLOPS/hr) is as follows:

| Hour | Time  | Server Work (TFLOPs/hr) |
|------|-------|--------------------------------|
| 0    | 9 AM  | 150                            |
| 1    | 10 AM | 250                            |
| 2    | 11 AM | 400                            |
| 3    | 12 PM | 500                            |
| 4    | 1 PM  | 500                            |
| 5    | 2 PM  | 450                            |
| 6    | 3 PM  | 350                            |
| 7    | 4 PM  | 200                            |

The additional heat (measured in Celsius) contributed by working is equal to  $.004$ per TFLOP.  


#### Ambient Temperature
At the same time, the room temperature is affected by the ambient environmental temperature.  Here is a forecasted temperature profile for a typical day:

| Hour | Time  | Ambient Temperature (°C) |
|------|-------|--------------------------|
| 0    | 9 AM  | 22                       |
| 1    | 10 AM | 24                       |
| 2    | 11 AM | 26                       |
| 3    | 12 PM | 28                       |
| 4    | 1 PM  | 30                       |
| 5    | 2 PM  | 31                       |
| 6    | 3 PM  | 30                       |
| 7    | 4 PM  | 28                       |

We will assume that the additional heat added to the room during hour $h$ is equal to
$$
.1 * ( \text{Ambient Temperature}_h- \text{Temperature in Room}_h).
$$
This value can be positive or negative (e.g. if it's cold outside.)  The constant $0.1$ represents the natural heat exchange of the building (depending on insulation, etc.)

#### Cooling Effort
Finally, we can exert effort to cool the room (cranking up the AC). For every kilowatt of Cooling Effort we exert in hour $h$, we expect the temperature to decrease by $0.5$ degrees centigrade.


Your goal is to write a linear optimization problem that minimizes the amount of cooling effort needed to keep the server room at a temperature of at most $25$ degrees Celsius throughout the day. Assume the server room starts at a temperature of $22$  degrees at the beginning of the day.



### Part a) Decision Variables (0 points)
I'm going to help you out a bit to get started.  We're going to us the following decision variables:
- Let $T_h$ be the room temperature (Celsius) in hour $h$, for $h = 0, \ldots, 7$.
- Let $C_h$ be the amount of cooling effort (in kilowatts) we apply in hour $h$, for $h = 0, \ldots, 7$.

So we have $16$ decision variables.  Make sure you count them and understand them.

*Teaching Note:  Note that $T_h$ is a bit weird as a decision-variable.  We don't really "decide" it... We decide $C_h$ and $T_h$ seems to be a consequence of the thermodynamic equations above.  But just trust me. We're going to need this extra decision-variable to make things work.*

T_0: Temperature at hour 0 of the day (units: degrees Celsius)
T_1: Temperature at hour 1 of the day (units: degrees Celsius)
T_2: Temperature at hour 2 of the day (units: degrees Celsius)
T_3: Temperature at hour 3 of the day (units: degrees Celsius)
T_4: Temperature at hour 4 of the day (units: degrees Celsius)
T_5: Temperature at hour 5 of the day (units: degrees Celsius)
T_6: Temperature at hour 6 of the day (units: degrees Celsius)
T_7: Temperature at hour 7 of the day (units: degrees Celsius)
T_8: Temperature at hour 8 of the day (units: degrees Celsius)

C_0: Cooling effect applied at hour 0 of the day (units: degrees Celsius reduction)
C_1: Cooling effect applied at hour 1 of the day (units: degrees Celsius reduction)
C_2: Cooling effect applied at hour 2 of the day (units: degrees Celsius reduction)
C_3: Cooling effect applied at hour 3 of the day (units: degrees Celsius reduction)
C_4: Cooling effect applied at hour 4 of the day (units: degrees Celsius reduction)
C_5: Cooling effect applied at hour 5 of the day (units: degrees Celsius reduction)
C_6: Cooling effect applied at hour 6 of the day (units: degrees Celsius reduction)
C_7: Cooling effect applied at hour 7 of the day (units: degrees Celsius reduction)

### Part b) (5 points):

What is the objective function for this problem? (Add a markdown cell directly below this one with your answer.) Describe the objective function both in words and in math.  include units.

The objective of this optimization model is to minimize the total cooling applied throughout the day. The model adjusts the cooling effect at each hour to achieve an optimal temperature regulation strategy while using the least amount of cooling possible.

Mathematical Representation:
Let  C_t  represent the cooling effect applied at hour  t . The objective function is:


$$ \sum_{t=0}^{7} C_t $$

This means the model seeks to find values for  C_0, C_1, $\dots$, C_7  that minimize their total sum while still meeting any constraints imposed by the problem (e.g., keeping the server room at a desired temperature range).

### Part c) (15 points)
What are the constraints for this problem? For each constraint, be sure to include a short description (in words) of what the constraint represents.  Your description should make the units clear. You should also relax any integrality constraints.  

**Hint:  Can you write an equation using the decision variables that tells you the temperature in the room at hour $1$ in terms of the temperature in hour $0$, the ambient temperature at hour $0$, and the cooling effort you put in at hour $0$?**


The room temperature at the start of the day is set to **22°C**.

  $$
  T_0 = 22
  $$

The room temperature cannot exceed **25°C** at any hour.

  $$
  T_h \leq 25, \quad \forall h \in \{1,2,3,4,5,6,7,8\}
  $$


The cooling effort at any hour must be **non-negative**.


$$
C_h \geq 0, \quad \forall h \in \{0,1,2,3,4,5,6,7\}
$$

The temperature at each hour depends on:
- The previous hour's temperature.
- The additional heat from **workload processing**, where each **TFLOP** adds **0.004°C**.
- The **ambient temperature** contribution, which adds **0.1 * (ambient temperature - room temperature)**.
- The **cooling effect**, which decreases the temperature by **0.5°C per unit of cooling effort**.

(for each hour \( h \)):  
  $$
  T_h = T_{h-1} + (0.004 \times \text{TFLOP}_h) + (0.1 \times (\text{Ambient}_h - T_{h-1})) - (0.5 \times C_{h-1})
  $$

  Expanding for each hour:

  $$
  T_1 = T_0 + (0.004 \times 150) + (0.1 \times (22 - T_0)) - (0.5 \times C_0)
  $$

  $$
  T_2 = T_1 + (0.004 \times 250) + (0.1 \times (24 - T_1)) - (0.5 \times C_1)
  $$

  $$
  T_3 = T_2 + (0.004 \times 400) + (0.1 \times (26 - T_2)) - (0.5 \times C_2)
  $$

  $$
  T_4 = T_3 + (0.004 \times 500) + (0.1 \times (28 - T_3)) - (0.5 \times C_3)
  $$

  $$
  T_5 = T_4 + (0.004 \times 500) + (0.1 \times (30 - T_4)) - (0.5 \times C_4)
  $$

  $$
  T_6 = T_5 + (0.004 \times 450) + (0.1 \times (31 - T_5)) - (0.5 \times C_5)
  $$

  $$
  T_7 = T_6 + (0.004 \times 350) + (0.1 \times (30 - T_6)) - (0.5 \times C_6)
  $$

  $$
  T_8 = T_7 + (0.004 \times 200) + (0.1 \times (28 - T_7)) - (0.5 \times C_7)
  $$



### Part d) (10 points)
Using your formulation above, code up your model in Gurobi and solve it.  Write the optimal value and optimal solution in a markdown cell directly below this one.  Be sure to label which is which and what the units are!

Below that markdown cell, include your python code for the model in one or more python cells.  Your code should print out the optimal value and optimal solution from your model as its last step. Code that does not run earns no credit!

In [7]:
#create model m3
m3 = Model("cooling")

In [8]:
# Create decision variables for the temperature every hour throughout day
T_0 = m3.addVar(name="T_0", vtype=GRB.CONTINUOUS)
T_1 = m3.addVar(name="T_1", vtype=GRB.CONTINUOUS)
T_2 = m3.addVar(name="T_2", vtype=GRB.CONTINUOUS)
T_3 = m3.addVar(name="T_3", vtype=GRB.CONTINUOUS)
T_4 = m3.addVar(name="T_4", vtype=GRB.CONTINUOUS)
T_5 = m3.addVar(name="T_5", vtype=GRB.CONTINUOUS)
T_6 = m3.addVar(name="T_6", vtype=GRB.CONTINUOUS)
T_7 = m3.addVar(name="T_7", vtype=GRB.CONTINUOUS)
T_8 = m3.addVar(name="T_8", vtype=GRB.CONTINUOUS)

# Create decision variables for the cooling effect at each hour throughout the day
C_0 = m3.addVar(name="C_0", vtype=GRB.CONTINUOUS)
C_1 = m3.addVar(name="C_1", vtype=GRB.CONTINUOUS)
C_2 = m3.addVar(name="C_2", vtype=GRB.CONTINUOUS)
C_3 = m3.addVar(name="C_3", vtype=GRB.CONTINUOUS)
C_4 = m3.addVar(name="C_4", vtype=GRB.CONTINUOUS)
C_5 = m3.addVar(name="C_5", vtype=GRB.CONTINUOUS)
C_6 = m3.addVar(name="C_6", vtype=GRB.CONTINUOUS)
C_7 = m3.addVar(name="C_7", vtype=GRB.CONTINUOUS)

# add initial constraint for room temperature at start of day
m3.addConstr(T_0 == 22, "Initial Temp")

# max temperature for each hour variable
m3.addConstr(T_1 <= 25, "MaxTemp1")
m3.addConstr(T_2 <= 25, "MaxTemp2")
m3.addConstr(T_3 <= 25, "MaxTemp3")
m3.addConstr(T_4 <= 25, "MaxTemp4")
m3.addConstr(T_5 <= 25, "MaxTemp5")
m3.addConstr(T_6 <= 25, "MaxTemp6")
m3.addConstr(T_7 <= 25, "MaxTemp7")
m3.addConstr(T_8 <= 25, "MaxTemp8")

# ensure non-negativity for cooling effect
m3.addConstr(C_0 >=0, "MaxCool0")
m3.addConstr(C_1 >=0, "MaxCool1")
m3.addConstr(C_2 >=0, "MaxCool2")
m3.addConstr(C_3 >=0, "MaxCool3")
m3.addConstr(C_4 >=0, "MaxCool4")
m3.addConstr(C_5 >=0, "MaxCool5")
m3.addConstr(C_6 >=0, "MaxCool6")
m3.addConstr(C_7 >=0, "MaxCool7")

# temperature constraints for each hour
# each temperature in the next hour depends on the temperature in the previous hour adjusted for...
# The additional heat (measured in Celsius) contributed by working is equal to $.004$ per TFLOP
# The TFLOP demand per hour is given in the table
# Additional heat added to room equals .1 * (ambient temperature_h - temperature in room_h)
# We are given the ambient temperature in the table
# For every kilowatt of Cooling Effort we exert in the hour, we expect the temperature to decrease by 0.5 degrees C.
m3.addConstr(T_1 == T_0 + (0.004 * 150) + (0.1 * (22 - T_0)) - (0.5 * C_0))
m3.addConstr(T_2 == T_1 + (0.004 * 250) + (0.1 * (24 - T_1)) - (0.5 * C_1))
m3.addConstr(T_3 == T_2 + (0.004 * 400) + (0.1 * (26 - T_2)) - (0.5 * C_2))
m3.addConstr(T_4 == T_3 + (0.004 * 500) + (0.1 * (28 - T_3)) - (0.5 * C_3))
m3.addConstr(T_5 == T_4 + (0.004 * 500) + (0.1 * (30 - T_4)) - (0.5 * C_4))
m3.addConstr(T_6 == T_5 + (0.004 * 450) + (0.1 * (31 - T_5)) - (0.5 * C_5))
m3.addConstr(T_7 == T_6 + (0.004 * 350) + (0.1 * (30 - T_6)) - (0.5 * C_6))
m3.addConstr(T_8 == T_7 + (0.004 * 200) + (0.1 * (28 - T_7)) - (0.5 * C_7))

# objective to minimize the total amount of cooling
m3.setObjective(C_0 + C_1 + C_2 + C_3 + C_4 + C_5 + C_6 + C_7, GRB.MINIMIZE)

# Optimize the model
m3.optimize()

# Print results
if m3.status == GRB.OPTIMAL:
    print("Optimal solution found:")
    print(f"Hour 0: Temperature = {T_0.x} °C, Cooling effort = {C_0.x} kW")
    print(f"Hour 1: Temperature = {T_1.x} °C, Cooling effort = {C_1.x} kW")
    print(f"Hour 2: Temperature = {T_2.x} °C, Cooling effort = {C_2.x} kW")
    print(f"Hour 3: Temperature = {T_3.x} °C, Cooling effort = {C_3.x} kW")
    print(f"Hour 4: Temperature = {T_4.x} °C, Cooling effort = {C_4.x} kW")
    print(f"Hour 5: Temperature = {T_5.x} °C, Cooling effort = {C_5.x} kW")
    print(f"Hour 6: Temperature = {T_6.x} °C, Cooling effort = {C_6.x} kW")
    print(f"Hour 7: Temperature = {T_7.x} °C, Cooling effort = {C_7.x} kW")
    print(f"Hour 8: Temperature = {T_8.x} °C")
    print(f"Total cooling effort: {m3.objVal} kW")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.4.0 23E224)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 25 rows, 17 columns and 41 nonzeros
Model fingerprint: 0x2782931e
Coefficient statistics:
  Matrix range     [5e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 2e+01]
Presolve removed 25 rows and 17 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.1532000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  2.153200000e+01
Optimal solution found:
Hour 0: Temperature = 22.0 °C, Cooling effort = 0.0 kW
Hour 1: Temperature = 22.6 °C, Cooling effort = 0.0 kW
Hour 2: Temperature = 23.740000000000002 °C, Cooling effort = 1.1320000000000032 kW
Hour 3: Temperature = 25.

### part e) (4 points)
In actuality, the price of electricity changes throughout the day.  In particular, it is more expensive in the afternoon when it is hot (because everyone turns on their AC and demand is high but supply is limited.)  Suppose you were given a forecast of the hourly price of electricity (similar to the forecasted temperatures).  How would you incorporate this information into your model to improve it? Does  it affect the decision variables, constraints, or objective? Describe the change in detailed paragraph.  

You do NOT need to reimplement your model.  A well-written description is enough.

#### change objective function
If we incorporated the hourly price of electricity, we would have to change the objective function to minimize the price of cooling that we are spending per hour. The units will change from total kilowatts to total cost of kilowatts in dollars. 


#### dont change variables or constraints
We wouldn't change the constraints because they are all based on the temperature, and the cost of cooling doesn't change the temperature. We wouldn't change the decision variables since the costs are given to us as constants.